In [6]:
from dotenv import load_dotenv

In [7]:
from langchain_groq import ChatGroq

In [8]:
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [9]:
llm.invoke("hi")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 4, 'total_tokens': 18, 'completion_time': 0.062667883, 'prompt_time': 0.00966934, 'queue_time': 0.042306369, 'total_time': 0.072337223}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--95c2350f-5c5f-45d9-987b-96ad2bf39d1d-0', usage_metadata={'input_tokens': 4, 'output_tokens': 14, 'total_tokens': 18})